In [1]:
na = (In=1:9, Fl=1.0:9)

(In = 1:9, Fl = 1.0:1.0:9.0)

In [7]:
# At compile time, the compiler does not know that the `Symbol` is `:Char`.
# It compiles getproperty(ta, ::Symbol), which returns `Vector{Any}`.
@code_warntype na.In

MethodInstance for getproperty(::NamedTuple{(:ab, :cd, :ef), Tuple{Vector{Int64}, Vector{Float64}, Vector{Bool}}}, ::Symbol)
  from getproperty(x, f::Symbol) in Base at Base.jl:38
Arguments
  #self#::Core.Const(getproperty)
  x::NamedTuple{(:ab, :cd, :ef), Tuple{Vector{Int64}, Vector{Float64}, Vector{Bool}}}
  f::Symbol
Body::Union{Vector{Bool}, Vector{Float64}, Vector{Int64}}
1 ─      nothing
│   %2 = Base.getfield(x, f)::Union{Vector{Bool}, Vector{Float64}, Vector{Int64}}
└──      return %2



In [8]:
# At compile time, the compiler knows that the `Symbol` is `:Char`.
# It compiles getproperty(ta, :Char), which returns `Vector{Char}`.
@code_warntype (na->na.In)(na)

MethodInstance for (::var"#1#2")(::NamedTuple{(:ab, :cd, :ef), Tuple{Vector{Int64}, Vector{Float64}, Vector{Bool}}})
  from (::var"#1#2")(na) in Main at In[8]:3
Arguments
  #self#::Core.Const(var"#1#2"())
  na::NamedTuple{(:ab, :cd, :ef), Tuple{Vector{Int64}, Vector{Float64}, Vector{Bool}}}
Body::Union{}
1 ─     Base.getproperty(na, :In)
└──     Core.Const(:(return %1))



If the compiler knows the columns extracted from a `NamedTuple` at compile time, the compiler optimizes code and return type.

In [4]:
na = (ab=[1, 2], cd=[3.0, 4.0], ef=[true, false])

# Luffy is the son of Dragon.

luffy(na, id) = na[id]

dragon(na) = luffy(na, 1)

;

In [5]:
@code_warntype luffy(na, 1)

MethodInstance for luffy(::NamedTuple{(:ab, :cd, :ef), Tuple{Vector{Int64}, Vector{Float64}, Vector{Bool}}}, ::Int64)
  from luffy(na, id) in Main at In[4]:5
Arguments
  #self#::Core.Const(luffy)
  na::NamedTuple{(:ab, :cd, :ef), Tuple{Vector{Int64}, Vector{Float64}, Vector{Bool}}}
  id::Int64
Body::Union{Vector{Bool}, Vector{Float64}, Vector{Int64}}
1 ─ %1 = Base.getindex(na, id)::Union{Vector{Bool}, Vector{Float64}, Vector{Int64}}
└──      return %1



In [6]:
@code_warntype dragon(na)

MethodInstance for dragon(::NamedTuple{(:ab, :cd, :ef), Tuple{Vector{Int64}, Vector{Float64}, Vector{Bool}}})
  from dragon(na) in Main at In[4]:7
Arguments
  #self#::Core.Const(dragon)
  na::NamedTuple{(:ab, :cd, :ef), Tuple{Vector{Int64}, Vector{Float64}, Vector{Bool}}}
Body::Vector{Int64}
1 ─ %1 = Main.luffy(na, 1)::Vector{Int64}
└──      return %1



It is not enough to have a type-stable data structure but also type-stable code.

`Base.getproperty(na, :In)` propagates `Constant` `In`, and the compiler knows the types.

`Base.getindex(na, id)` does not tell about `id`'s type, and the compiler must figure it out and optimize during run time.